In [ ]:
import ee

ee.Authenticate()
# Initialize the Earth Engine API
ee.Initialize()

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://code.earthengine.google.com/client-auth?scopes=https%3A//www.googleapis.com/auth/earthengine%20https%3A//www.googleapis.com/auth/devstorage.full_control&request_id=473faQI3sCSP8Mx6RUEZLpQRgKxiCxiefkiDfELuA2w&tc=S2cxDPkq1SXQ66d7hKXTZOqFJAG2mK43tE2YL7rJRso&cc=jJdMSq01expMV339VUb5rvc00Grat-ltbAQ7RUt_hWo

The authorization workflow will generate a code, which you should paste in the box below.
Enter verification code: 4/1AfJohXmYnaq-WIOCaprF9IANocY5QcuVIDIYVkUJxMnDGivB1Y7kqwe_O2E

Successfully saved authorization token.


In [ ]:
import openpyxl

# Create a new Excel workbook
#workbook = openpyxl.Workbook()

# Get the active sheet (default sheet created with the workbook)
#sheet = workbook.active

# Write data to the sheet
#sheet['A1'] = 'Lon'
#sheet['B1'] = 'Lat'
#sheet['C1'] = 'Elevation'
#sheet['D1'] = 'Distance'

distances = []
final_points = []
#g = open("Full.txt", "w")
f = open("Full.txt", "r")
prev = 0
i = 0
for x in f:
  strs = x.split('\t')
  #print(strs)
  distance = float(strs[2].split()[1])
  #distance += prev
  #if(i > 1 and distance < distances[i - 1]):
    #prev += distances[i - 1]
  elevation = int(strs[1].split()[1])
  coords_raw = strs[0].split()
  coords = [float(coords_raw[1]), float(coords_raw[2])]
  #pressure_loss = float(strs[4].split()[1])
  #sheet[f"A{i}"] = coords[0]
  #sheet[f"B{i}"] = coords[1]
  #sheet[f"C{i}"] = elevation
  #sheet[f"D{i}"] = distance
  string = f"coords: {coords_raw[1]} {coords_raw[2]}\t elevation: {elevation}\t distance: {distance}\n"
  #g.write(string)
  final_points.append(coords)
  distances.append(distance)
  i += 1

#g.close()
f.close()
#workbook.save('fullExcel.xlsx')
#print(final_points)
l = len(distances)
print(distances[l - 1])


1556.4556621311208


In [ ]:
import geemap
import folium

# Create a Map
Map = geemap.Map()

startPoint = [(-0.05 * 8) + 31.351498594, 1.42583163]     #Hoima
#endPoint = [32.403194444, -2.87491666]    #Temp

#startPoint = [32.3914985940, -2.8141683700]  #temp
endPoint = [39.105554, -5.066 - 15 * 0.08]

rq = [startPoint[0] - 0.5, startPoint[1] + 0.5, endPoint[0] + 0.5, endPoint[1] - 0.5]
rq = ee.Geometry.Rectangle(rq)
# Add the datasets to the map
surface_water_image = geemap.ee.Image('JRC/GSW1_4/GlobalSurfaceWater').clip(rq)
wdpa = geemap.ee.FeatureCollection('WCMC/WDPA/current/polygons').filterBounds(rq)
line = geemap.ee.Geometry.LineString(final_points)

#Pump stations
pump_sattions_raw = [
    [31.341498594, 1.42583163],
    [31.581498594, -1.69416837],
    [32.181498594, -2.57416837],
    [34.251498594, -4.81416837],
    [37.131498594, -5.37416837],
    [38.871498594, -5.21416837]
]

pump_stations = []
for i in pump_sattions_raw:
  temp_pump = ee.Geometry.Point(i)
  #temp_pump = temp_pump.buffer(100)
  pump_stations.append(temp_pump)
pump_stations = ee.FeatureCollection(pump_stations)

# Display the datasets on the map
Map.addLayer(surface_water_image, {}, 'Global Surface Water')
Map.addLayer(wdpa, {}, 'WDPA Polygons')
Map.addLayer(geemap.ee.Feature(line), {'color': 'blue'}, 'Line')
Map.addLayer(pump_stations, {'color': 'green'}, 'Pump stations')


# Center the map
Map.centerObject(wdpa, 4)
# Save the map as an HTML file
html_file_path = "output_map.html"
Map.to_html(html_file_path)
# Display the map
Map

Map(center=[-3.6614019908426125, 34.40272752039558], controls=(WidgetControl(options=['position', 'transparent…